In [1]:
import numpy as np

from framework_pkg.framework import FrameWork
from framework_pkg.framework import bin_prediction

from cobaya.run import run

import warnings
warnings.filterwarnings('ignore')


Warning!: Python module pyslha not found, disabling slha reading routines


In [2]:
param ={'SinT12'  : 0.308 ,
        'T13'  : 8.57, 
        'mu1'  : 0.,
        'mu2'  : 0., 
        'mu3'  : 0.  ,
        'mdm'  : 9.02e-4 , 
        'alpha': 0.  ,
        'eps' : 0.  ,
        'alpha_eps' : 0.  ,
        'M12'  : 6.9e-5 }

fw_modulation = FrameWork(threshold=4.5, bin_max=13, lat=36, efficiency_correction=True, resolution_correction=False)
total_days = fw_modulation.time_day

exposure = np.loadtxt('./Data/time_exposures_1.txt') 

data = np.loadtxt('./Data/modulation_data.txt')
sigma_p = 0.014
mean_data = np.mean(data[:,1])

In [3]:
def LogLikelihood(sint12, m12, mdm, mu2, alpha, eps, alpha_eps, delta_p):
    param['SinT12'] = sint12
    param['M12'] = m12 * 1e-5

    param['mu2'] = mu2
    param['alpha'] = alpha
    param['eps'] = eps
    param['alpha_eps'] = alpha_eps

    param['mdm'] = mdm * 2e-4

    raw_result = fw_modulation.__getitem__(param) 
    _, _, R_bin_total = bin_prediction(raw_result, exposure, total_days)
        
    prediction = R_bin_total[:, 0] * ( 1 - delta_p * sigma_p)
    prediction = prediction - np.mean(prediction)

    cond1 = data[:,1] - mean_data >= prediction
    cond2 = data[:,1] - mean_data < prediction
    lower_part  = (data[cond1,1] - mean_data - prediction[cond1])**2 / data[cond1,3]**2
    upper_part = (data[cond2,1] - mean_data - prediction[cond2])**2 / data[cond2,2]**2

    return -0.5 * ( np.sum(lower_part) + np.sum(upper_part) + delta_p**2 ) 

In [4]:
print(LogLikelihood(0.308, 7.22, 4.97, 0., 0., 0., 0., 0.))

print(LogLikelihood(0.300, 7.66, 5.3, 0.127, 0.945, 0.827, 2.33, 0.25))

-422.829180202607
-413.3949230649136


In [10]:
info = {"likelihood": {"Chi2": LogLikelihood},
        
        "params": dict([("sint12" , {"prior" : {"min": 0.275, "max": 0.345},
                                  "latex" : r"\sin(\theta_{12})^2"  
                                  }),

                        
                        ("m12"    , {"prior" : {"min": 6.92, "max": 8.05},
                                  "latex": r"\Delta m^2_{21} \ \rm 10^{-5} \ eV^2"  
                                  }), 
                        
                        ("mu2"    , {"prior" : {"min": 0, "max": 0.25},
                                   "latex" : r"\tilde{\mu}^2/2 "  
                                   }),
                        
                        ("alpha"  , {"prior" : {"min": 0, "max": np.pi},
                                   "latex" : r"\alpha "  
                                   }),
                        
                        ("eps"  , {"prior" : {"min": 0, "max": 1.},
                                   "latex" : r"\epsilon "  
                                   }),

                        ("alpha_eps"  , {"prior" : {"min": 0, "max": np.pi},
                                   "latex" : r"\alpha_\epsilon "  
                                   }), 

                        ("delta_p", {"prior" : {"dist": 'norm', "loc": 0, "scale": 1},
                                  "latex": r"\delta_p"  
                                  }) ,

                        ("mdm", {"prior" : {"min": 1, "max": 10},
                                   "latex": r"m_{\rm dm}"  
                                  }) 
                        
                        ]),
        
        "sampler": {"mcmc": {"Rminus1_stop": 0.01, "max_tries": 100000 }},
        "output" : "output8/run_info"
       }

In [ ]:
#updated_info,sampler = run(info)